In [1]:
import numpy as np
import os
import math
import random
import torch
# from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from scipy import stats
import plotly.graph_objects as go

from sklearn.utils import shuffle
import torch.nn as nn

from sklearn import metrics

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms

USE_CUDA = True

In [3]:
dataset = pd.read_csv('../capsule_code/data.csv')
dataset

,Unnamed: 0,0,1,2,3,4,5,6,label
0,0,-1.301023,-1.651728,-1.538896,-1.562533,-0.329070,0.350222,-0.391974,11
1,1,-1.301023,-1.651728,-1.538896,-1.562533,-0.329070,0.350222,-0.391974,11
2,2,-1.301023,-1.651728,-1.538896,-1.562533,-0.329070,0.350222,-0.391974,11
3,3,-1.301023,-1.651728,-1.538896,-1.562533,-0.329070,0.350222,-0.391974,11
4,4,-1.301023,-1.651728,-1.538896,-1.275513,-0.329070,0.348971,-0.428100,11
...,...,...,...,...,...,...,...,...,...
11995,11995,1.181845,1.620713,1.450329,1.594679,-0.368392,-0.005276,-0.428100,14
11996,11996,1.181845,1.620713,1.586202,-1.562533,-0.368392,-0.004337,-0.428100,0
11997,11997,1.181845,1.620713,1.586202,-1.275513,-0.368392,-0.004963,-0.428100,0
11998,11998,1.181845,1.620713,1.586202,0.159583,-0.368392,-0.005901,-0.428100,0


In [4]:
all_data = shuffle(dataset)

In [5]:
data = all_data[['0','1','2','3','4','5','6']]
label = all_data['label']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state=1) # 0.25 x 0.8 = 0.2
X_train, X_test, X_valid = np.array(X_train).reshape(-1, 7), np.array(X_test).reshape(-1, 7), np.array(X_valid).reshape(-1, 7)
y_train, y_test, y_valid = np.array(y_train).reshape(-1, 1), np.array(y_test).reshape(-1, 1), np.array(y_valid).reshape(-1, 1)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train, X_valid, X_test = torch.FloatTensor(X_train), torch.FloatTensor(X_valid), torch.FloatTensor(X_test)
y_train, y_valid, y_test = torch.FloatTensor(y_train), torch.FloatTensor(y_valid), torch.FloatTensor(y_test)

In [8]:
config = {
    'batch_size': 50,
    'shuffle': True,
    'num_workers': 6,
    'drop_last': False,
    'num_epochs': 50,
#     'encode_dim': 1,
#     'hidden_dim': 128,
#     'output_dim': 1,
#     'num_layers': 3,
    'dropout': 0.5,
    'device': device
}

In [9]:
from torch.utils.data import Dataset, DataLoader

class Dataset(Dataset):
  # 'Characterizes a dataset for PyTorch'
    def __init__(self, data, labels):
        # 'Initialization'
        self.labels = labels
        self.data = data

    def __len__(self):
        # 'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        # 'Generates one sample of data'
        # Select sample
        return self.data[index], self.labels[index]

In [10]:
train_set = Dataset(X_train, y_train)
train_loader = DataLoader(
    train_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

validation_set = Dataset(X_valid, y_valid)
validation_loader = DataLoader(
    validation_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

test_set = Dataset(X_test, y_test)
test_loader = DataLoader(
    test_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

# 1.使用CNN训练异常分类器

In [20]:
class CNN(nn.Module):
    def __init__(self):
        
        super(CNN, self).__init__()
        
        # x: torch.Size([32, 7, 1])
        # torch.Size([32, 16, 3])
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels = 7, out_channels = 64, kernel_size = 3, stride = 1, padding = 2), 
            nn.BatchNorm1d(64),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.2)
        )
        
        # torch.Size([32, 32, 5])
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 2, stride = 1, padding = 1), 
            nn.BatchNorm1d(32),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.2)
        )
#         # torch.Size([32, 64, 7])
#         self.layer3 = nn.Sequential(
#             nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 2, stride = 1, padding = 1), 
#             nn.BatchNorm1d(64),
#             nn.LeakyReLU(inplace=True),
#             nn.Dropout(p=0.5)
#         )
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(True),
            nn.Linear(64,16)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
#         x = self.layer3(x)
#         print("x3:",x.shape)
#         x = self.layer4(x)
#         print("x4:",x.shape)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
#         return F.log_softmax(self.l5(x), dim=1)
        return x

In [13]:
cnn = CNN()
cnn

CNN(
  (layer1): Sequential(
    (0): Conv1d(7, 64, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv1d(64, 32, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Dropout(p=0.2, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=True)
    (3): Linear(in_features=64, out_features=16, bias=True)
    (4): Softmax(dim=1)
  )
)

In [14]:
for batch_i, (batch_data, batch_labels) in enumerate(train_loader):
    batch_data = batch_data.unsqueeze(0)
    batch_data = batch_data.permute(1,2,0)
    output = cnn(batch_data)

In [15]:
output.shape, output

(torch.Size([50, 16]),
 tensor([[0.0643, 0.0727, 0.0459, 0.0377, 0.0955, 0.0831, 0.0940, 0.0614, 0.0429,
          0.0572, 0.0463, 0.0676, 0.0729, 0.0505, 0.0436, 0.0644],
         [0.0674, 0.0383, 0.0720, 0.0496, 0.0685, 0.1158, 0.0495, 0.0692, 0.0518,
          0.0894, 0.0573, 0.0678, 0.0489, 0.0319, 0.0661, 0.0565],
         [0.0920, 0.0551, 0.0512, 0.0433, 0.0701, 0.1097, 0.0405, 0.0553, 0.0382,
          0.0981, 0.0687, 0.0870, 0.0400, 0.0342, 0.0715, 0.0450],
         [0.0751, 0.0542, 0.0776, 0.0733, 0.0411, 0.0401, 0.0657, 0.1110, 0.0792,
          0.0518, 0.0217, 0.0443, 0.0296, 0.1466, 0.0328, 0.0560],
         [0.0277, 0.0595, 0.0332, 0.0927, 0.0670, 0.0415, 0.0374, 0.0805, 0.1207,
          0.0370, 0.0953, 0.0746, 0.1100, 0.0181, 0.0704, 0.0344],
         [0.0292, 0.0140, 0.0539, 0.0244, 0.0154, 0.0260, 0.0396, 0.0160, 0.0282,
          0.0315, 0.0529, 0.0173, 0.0099, 0.0074, 0.0917, 0.5424],
         [0.0594, 0.0467, 0.0690, 0.0756, 0.0600, 0.0614, 0.0768, 0.0535, 0.1007,
 

In [16]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
def One_hot(labels):
    num_class=16
    lb=LabelBinarizer().fit(np.array(range(num_class)))
    labels=lb.transform(labels)
    return labels

In [17]:
criterion = nn.CrossEntropyLoss()  #使用交叉熵作为损失函数
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
num_epochs = 50
num_classes = 16
learning_rate = 0.0001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# 225
total_step = len(train_loader)
print("total_step:", total_step)
loss_list = []
acc_list = []

# writer = SummaryWriter()

total_step: 144


In [21]:
tf_iter = 0 
batch_size_train = 1
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.unsqueeze(0)
        inputs = inputs.permute(1,2,0)
        labels = labels.squeeze()
        labels = labels.type(torch.LongTensor)
        
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # 反向传播和优化器
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 精度
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        accuracy = (correct / total) * 100
        
#         writer.add_scalar('Accuracy/train',  accuracy,  tf_iter)
#         writer.add_scalar('Loss/train',  loss.item(),  tf_iter)
        tf_iter += 1
        
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, (i + 1)*batch_size_train,
                  total_step, loss.item(),
                  accuracy))
    
print("----------------模型经过训练!-------------------- ")

Epoch [1/50], Step [50/144], Loss: 2.1384, Accuracy: 74.00%
Epoch [1/50], Step [100/144], Loss: 2.0981, Accuracy: 78.00%
Epoch [2/50], Step [50/144], Loss: 2.0321, Accuracy: 84.00%
Epoch [2/50], Step [100/144], Loss: 2.1178, Accuracy: 76.00%
Epoch [3/50], Step [50/144], Loss: 2.3726, Accuracy: 50.00%
Epoch [3/50], Step [100/144], Loss: 2.1820, Accuracy: 70.00%
Epoch [4/50], Step [50/144], Loss: 2.2578, Accuracy: 62.00%
Epoch [4/50], Step [100/144], Loss: 2.0172, Accuracy: 86.00%
Epoch [5/50], Step [50/144], Loss: 2.1666, Accuracy: 72.00%
Epoch [5/50], Step [100/144], Loss: 2.1500, Accuracy: 72.00%
Epoch [6/50], Step [50/144], Loss: 2.1730, Accuracy: 70.00%
Epoch [6/50], Step [100/144], Loss: 2.1390, Accuracy: 74.00%
Epoch [7/50], Step [50/144], Loss: 2.0764, Accuracy: 80.00%
Epoch [7/50], Step [100/144], Loss: 2.2264, Accuracy: 66.00%
Epoch [8/50], Step [50/144], Loss: 2.1772, Accuracy: 70.00%
Epoch [8/50], Step [100/144], Loss: 2.1259, Accuracy: 76.00%
Epoch [9/50], Step [50/144], Los

In [19]:
print_every = 32
# 在训练集上循环训练
for epoch in range(2):
    # 损失值
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # 获得输入张量和标签
        inputs, labels = data
        inputs = inputs.unsqueeze(0)
        inputs = inputs.permute(1,2,0)
        labels = labels.squeeze()
        labels = labels.type(torch.LongTensor)
        # 模型梯度归零，否则会不断累积。optimizer.zero_grad()也可以
        cnn.zero_grad()
        # 数据经过模型计算
        labels = labels.squeeze()
        outputs = cnn(inputs)
        # 计算模型预测与实际标签的损失值
        loss = criterion(outputs, labels)
        # 梯度自动反向传播
        loss.backward()
        # 优化器执行优化，按梯度下降原则调整参数
        optimizer.step()
        
        running_loss += loss.item()
        losses = []
        # 每1000轮输出一次损失值
        if batch_i != 0 and batch_i % print_every == 0:
            avg_train_loss = running_loss/print_every
            losses.append(avg_train_loss)
            print('Epoch: {} \tTraining Loss: {:.8f}'.format(epoch, avg_train_loss))
            running_loss = 0 # reset accumulated training loss


In [19]:
from sklearn import metrics
# 训练后的模型测试效果
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
# 测试预测时取消自动梯度，即向前计算时不自动求导
with torch.no_grad():
    for data, labels in test_loader:
        data = data.unsqueeze(0)
        data = data.permute(1,2,0)
        outputs = cnn(data)
        labels = labels.data.numpy()
        predicted = torch.max(outputs.data, 1)[1].numpy()
        labels_all = np.append(labels_all, labels)
        predict_all = np.append(predict_all, predicted)

confusion = metrics.confusion_matrix(labels_all, predict_all)
print(u'混淆矩阵：')
print(confusion)
# 验证集准确度
acc = metrics.accuracy_score(labels_all, predict_all)
print(u'测试集的准确率: {0:.2%}'.format(acc))

混淆矩阵：
[[1179    0    1    0    0    0    3    1    0    0    0    7    0    0
     0    0]
 [   0   91    1    0    0    0    1    0    0    0    0    0    0    0
     0    0]
 [   1    1   56    4    0    6    0    0    7    6    0    0    0    0
     0    0]
 [   0    0    7   36    0   17    0    0    0    0    0    0    0    0
     0    0]
 [  93    0    0    0    0    2    0    0    0    0    0    0    0    0
     0    0]
 [   1    0    6   22    0   23    1    0   39    0    0    0    0    0
     1    0]
 [  17    3    0    0    0    1   53    0    6    0    0    1    0    0
     0    0]
 [  82    0    0    0    0    0    0    0    0    0    1    0    0    0
     0    0]
 [   0    0    4    0    0    0    1    0   62    0    0    0    0    0
     0    0]
 [   0    0   26    1    0    0    0    0    0   60    0    2    0    0
     0    0]
 [  70    0    0    0    0    0    1    0    0    0    0    0    0    0
     0    0]
 [  80    0    0    0    0    0    2    0    0    1    0   